# DATA CLEANING

In [18]:
import os
import nibabel as nib
from nibabel.testing import data_path
import numpy as np
import cv2 
from matplotlib import pyplot as plt 
import sys
import shutil

In [20]:
cond= "MCI" #condition is AD or CN or MCI
seq = "T2"
tesla = 3 #1.5 or 3
downloadDate = "12_05_2022"
meta_path = "./Metadata/{}w_{}_ADNI1_{}T_{}.csv".format(seq, cond, tesla, downloadDate)
path = "./{}T/{}/{}/".format(tesla, seq, cond)

In [21]:
def show_slices(slices):
    # """ Function to display row of image slices """
    fig, axes = plt.subplots(1, len(slices))
    for i, slice in enumerate(slices):
        axes[i].imshow(slice.T, cmap="gray", origin="lower")

In [22]:
def plot_slices(a, b, nii_image, main_title):
    fig = plt.figure(figsize=(20, 16))
    # Add a margin between the main title and sub-plots
    fig.subplots_adjust(top=0.93)
    fig.suptitle(str(main_title), fontsize = 16)
    num_rows= 8
    num_columns=10
    for i in range(a, b):
            # Adds a subplot at the 1st position
        fig.add_subplot(num_rows, num_columns, i-a+1)
        # showing image
        plt.imshow(np.rot90(nii_image[:, :, i]), cmap='gray')
        plt.axis('off')
        plt.title("Slice "+ str(i+1))

    fig.savefig("./data_cleaning/"+ main_title + ".jpg")

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
print(datetime.now().strftime('%Y%m%d-%H%M%S'))

meta_df = pd.read_csv(meta_path, delimiter=',')

20221220-140533


In [24]:
meta_df

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
0,I95396,002_S_0729,MCI,F,67,5,MRI,Double_TSE,Original,3/05/2008,DCM,NaN
1,I20092,002_S_0729,MCI,F,65,2,MRI,Double_TSE,Original,8/02/2006,DCM,12/05/2022
2,I71862,002_S_0729,MCI,F,66,4,MRI,Double_TSE,Original,9/05/2007,DCM,NaN
3,I153080,002_S_0729,MCI,F,68,8,MRI,Double_TSE,Original,8/13/2009,DCM,NaN
4,I118553,002_S_0729,MCI,F,67,6,MRI,Double_TSE,Original,9/29/2008,DCM,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1562,I1619413,941_S_7106,MCI,F,72,1,MRI,Axial 3TE T2 STAR,Original,9/09/2022,DCM,NaN
1563,I1619415,941_S_7106,MCI,F,72,1,MRI,Field Mapping,Original,9/09/2022,DCM,NaN
1564,I1619401,941_S_7106,MCI,F,72,1,MRI,Axial 3TE T2 STAR,Original,9/09/2022,DCM,NaN
1565,I1619410,941_S_7106,MCI,F,72,1,MRI,Sagittal 3D FLAIR,Original,9/09/2022,DCM,NaN


In [25]:
meta_dict = {} #dict

# meta_dict[0].append(meta_df.loc[:,"Image Data ID"])
meta_dict["ImageID"] = meta_df.loc[:,"Image Data ID"]
meta_dict["SubjectID"] = meta_df.loc[:,"Subject"]
meta_dict["Group"] = meta_df.loc[:,"Group"]
meta_dict["Sex"] = meta_df.loc[:,"Sex"]
meta_dict["Age"] = meta_df.loc[:,"Age"]
meta_dict["Visit"] = meta_df.loc[:,"Visit"]
meta_dict["Modality"] = meta_df.loc[:,"Modality"]
meta_dict["Description"] = meta_df.loc[:,"Description"]
meta_dict["Type"] = meta_df.loc[:,"Type"]
meta_dict["AcqData"] = meta_df.loc[:,"Acq Date"]
meta_dict["Format"] = meta_df.loc[:,"Format"]

# meta_series = meta_df.loc[:,"Series.ID"]
# meta_study = meta_df.loc[:,"Study.ID"]
meta_image = meta_df.loc[:,"Image Data ID"]
meta_subject = meta_df.loc[:,"Subject"]
meta_visit = meta_df.loc[:,"Visit"]

meta_combined= []
i=0
for i in range(0,len(meta_image)):
    # combined = str(meta_subject[i])+"-"+str(meta_series[i])+"-"+str(meta_image[i])
    combined = str(meta_subject[i])+"-"+str(meta_image[i])

    meta_combined.append(combined)

    i+=1

In [29]:
meta_dict.clear()

In [30]:
meta_dict = {"Image Data ID" : [], "Subject":[], "Group":[],"Sex":[],"Age":[],"Visit":[],"Modality":[],"Description":[],"Type":[],"Acq Date":[],"Format":[]}

# Preprocess if T1

In [ ]:
from pathlib import Path
# path = "C:/Users/Brain-LPDP/Documents/DEWINDA PMDSU/Publication/2023 - Research Visit KHU/Experiment/ADNI1 All Data/ADNI"
# path ="./ADNI/"
target_meta = "./TempMeta/"
target_path = "./TempData/"
# path = path.replace("\\","/")
# target_path = r'C:\Users\Brain-LPDP\Documents\AATEST\TESTDEST\\'
# src = r'C:\Users\Brain-LPDP\Documents\AATEST\TESTSRC'

result = list(Path(path).glob('**/*.nii'))
unique = set(result)
print(len(unique))
print(len(result))
# # print(result[0:5])
# print(mystr.split('\\')[-1])

i = 0
ctr = 0
for f in result[:]:
    # newFilename = str(f).partition("/ADNI_")[2]
    # print(newFilename)
    pathToFile = str(f)
    fileName = pathToFile.split("/")[7] #use "\\" if "/" does not work
    fileNameNoExt = fileName.split(".nii")[0]
    # print(fileName)
    
    part = fileNameNoExt.split('_MR')
    id_subject = part[0].split("ADNI_")[1]
    part = part[1].split("raw_")[1]
    # print(part)
    id_series = part.split("_S")[1]
    # print(id_series)
    id_series = id_series.split("_I")[0]
    id_image = fileNameNoExt.split("_I")[1]

   
    combinedIDs = id_subject+"-I"+id_image
    subdirName = id_subject+"-"+id_series+"-"+id_image
    
    # print(combinedIDs)
    # print(target_dir)    
    if "RAGE" in fileName or "rage" in fileName:
    # if "TSE" in fileName or "FSE" in fileName: #FOR T2 
        # print("YES")
        target_dir =os.path.join(target_path, cond)
        target_dir = os.path.join(target_dir, subdirName)
        isExist = os.path.exists(target_dir)    
    
        j = 0
        for j in range(len(meta_combined)):    
            if meta_visit[j] <= 2: 
                if meta_combined[j] in combinedIDs:
                    # Copy dataframe
                    print("HERE")
                    
                    meta_dict["Image Data ID"].append(meta_df.loc[j,"Image Data ID"])
                    meta_dict["Subject"].append(meta_df.loc[j,"Subject"])
                    meta_dict["Group"].append(meta_df.loc[j,"Group"])
                    meta_dict["Sex"].append(meta_df.loc[j,"Sex"])
                    meta_dict["Age"].append(meta_df.loc[j,"Age"])
                    meta_dict["Visit"].append(meta_df.loc[j,"Visit"])
                    meta_dict["Modality"].append(meta_df.loc[j,"Modality"])
                    meta_dict["Description"].append(meta_df.loc[j,"Description"])
                    meta_dict["Type"].append(meta_df.loc[j,"Type"])
                    meta_dict["Acq Date"].append(meta_df.loc[j,"Acq Date"])
                    meta_dict["Format"].append( meta_df.loc[j,"Format"])

                    # if isExist == False:
                    #     os.mkdir(target_dir)
                    # print(target_dir)
                    print("File {} copied to target folder".format(fileName))
                    # shutil.copy(f,target_dir+"/"+fileName)

                    i+=1
            j+=1
    ctr+=1    
    print(ctr)
print("Total {}w data is {}".format(seq, i))


# Preprocess if T2

In [ ]:
from pathlib import Path
# path = "C:/Users/Brain-LPDP/Documents/DEWINDA PMDSU/Publication/2023 - Research Visit KHU/Experiment/ADNI1 All Data/ADNI"
# path ="./ADNI/"
target_meta = "./TempMeta/"
target_path = "./TempData/"
# path = path.replace("\\","/")
# target_path = r'C:\Users\Brain-LPDP\Documents\AATEST\TESTDEST\\'
# src = r'C:\Users\Brain-LPDP\Documents\AATEST\TESTSRC'

result = list(Path(path).glob('**/*.nii'))
unique = set(result)
print(len(unique))
print(len(result))
# # print(result[0:5])
# print(mystr.split('\\')[-1])

i = 0
ctr = 0
for f in result[:]:
    # newFilename = str(f).partition("/ADNI_")[2]
    # print(newFilename)
    pathToFile = str(f)
    fileName = pathToFile.split("/")[7] #use "\\" if "/" does not work
    print(fileName)
    fileNameNoExt = fileName.split(".nii")[0]
    # print(fileName)
    
    part = fileNameNoExt.split('_MR')
    id_subject = part[0].split("ADNI_")[1]
    part = part[1].split("raw_")[1]
    # print(part)
    id_series = part.split("_S")[1]
    # print(id_series)
    id_series = id_series.split("_I")[0]
    id_image = fileNameNoExt.split("_I")[1]

    combinedIDs = id_subject+"-I"+id_image
    subdirName = id_subject+"-"+id_series+"-"+id_image
    
    # print(combinedIDs)
    # print(target_dir)    
    # if "RAGE" in fileName or "rage" in fileName:
    if "TSE" in fileName or "FSE" in fileName: #FOR T2 
        # print("YES")
        target_dir =os.path.join(target_path, cond)
        target_dir = os.path.join(target_dir, subdirName)
        isExist = os.path.exists(target_dir)    
    
        j = 0
        for j in range(len(meta_combined)):    
            if meta_visit[j] < 6: 
                if meta_combined[j] in combinedIDs:
                    # Copy dataframe
                    print("HERE")
                    
                    meta_dict["Image Data ID"].append(meta_df.loc[j,"Image Data ID"])
                    meta_dict["Subject"].append(meta_df.loc[j,"Subject"])
                    meta_dict["Group"].append(meta_df.loc[j,"Group"])
                    meta_dict["Sex"].append(meta_df.loc[j,"Sex"])
                    meta_dict["Age"].append(meta_df.loc[j,"Age"])
                    meta_dict["Visit"].append(meta_df.loc[j,"Visit"])
                    meta_dict["Modality"].append(meta_df.loc[j,"Modality"])
                    meta_dict["Description"].append(meta_df.loc[j,"Description"])
                    meta_dict["Type"].append(meta_df.loc[j,"Type"])
                    meta_dict["Acq Date"].append(meta_df.loc[j,"Acq Date"])
                    meta_dict["Format"].append( meta_df.loc[j,"Format"])

                    if isExist == False:
                        os.mkdir(target_dir)
                    print(target_dir)
                    print("File {} copied to target folder".format(fileName))
                    shutil.copy(f,target_dir+"/"+fileName)

                    i+=1
                j+=1
    ctr+=1    
    print(ctr)
print("Total {}w data is {}".format(seq, i))


In [32]:
import pandas as pd

# convert the history.history dict to a pandas DataFrame:     
temp_meta_df = pd.DataFrame(meta_dict) 

meta_csv_file = "./TempMeta/KHU_{}w_{}_ADNI1_3T_12_05_2022_Cleaned.csv".format(seq, cond)

with open(meta_csv_file, mode='w') as f:
    temp_meta_df.to_csv(f)

In [33]:
temp_meta_df

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format
0,I34482,005_S_0572,MCI,M,79,3,MRI,FSE PD/T2,Original,12/27/2006,DCM
1,I20025,005_S_0572,MCI,M,79,2,MRI,FSE PD/T2,Original,7/31/2006,DCM
2,I58877,005_S_0572,MCI,M,80,4,MRI,FSE PD/T2,Original,7/09/2007,DCM
3,I89321,005_S_0572,MCI,M,81,5,MRI,Axial PD/T2 FSE,Original,2/04/2008,DCM
4,I20782,130_S_0449,MCI,F,67,2,MRI,Double_TSE SENSE,Original,8/16/2006,DCM
...,...,...,...,...,...,...,...,...,...,...,...
164,I45312,068_S_1075,MCI,M,67,2,MRI,ADNI Double_TSE,Original,3/02/2007,DCM
165,I74927,136_S_0195,MCI,M,81,5,MRI,Double_TSE,Original,9/24/2007,DCM
166,I46775,136_S_0195,MCI,M,81,4,MRI,Double_TSE,Original,3/26/2007,DCM
167,I25483,136_S_0195,MCI,M,80,3,MRI,Double_TSE,Original,10/02/2006,DCM


In [13]:
len(meta_dict["Subject"])

153